# Work In Progress (WIP)

# 2. Quantum Logic Gates

In [1]:
import numpy as np

## What are quantum logic gates?

In [notebook 1](https://github.com/CarloLepelaars/q4p/blob/main/nbs/1-the-qubit.ipynb) we have firmly established the idea foundation of quantum computing as a state evolving through a series of quantum logic gates. We have also discussed one of the simplest meaningful logic gates called the X (NOT) gate. Here we will discuss the foundation of these gates and the most common implementations. We will start with a single qubit and then move on to multi-qubit gates.


Recall that a quantum state for a single qubit is a vector with 2 complex numbers. Almost all quantum algorithms start with the zero state:

$$
\begin{bmatrix}
1 \\
0
\end{bmatrix} = |0\rangle
$$

In [2]:
zero_state = np.array([1, 0], dtype=complex)
zero_state

array([1.+0.j, 0.+0.j])

## Pauli Gates

The X (NOT) gate discussed in notebook 1 is one of the fundamental Pauli Gates, which allow us to rotate the state around 3 axes. The have a special significance in quantum computing as they are all "observables", which means they can be used for measuring a qubit. Recall that in most quantum algorithms we measure in the "Z-basis". Lastly, for completenesswe define an Identity gate (I), which does not change the state. The Pauli Gates and Identity gate can be described as matrices:

$$
X = \sigma_x = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix},
\quad
Y = \sigma_y = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix},
\quad
Z = \sigma_z = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix},
\quad
I = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$


In [3]:
X = np.array([[0, 1], 
              [1, 0]], dtype=complex)

Y = np.array([[0, -1j], 
              [1j, 0]], dtype=complex)

Z = np.array([[1, 0], 
              [0, -1]], dtype=complex)

I = np.array([[1, 0], 
              [0, 1]], dtype=complex)

Rotating $| 0 \rangle$ around the Y-axis ($Y|0\rangle$) gives us the state $i|1\rangle$:

In [4]:
Y @ zero_state

array([0.+0.j, 0.+1.j])

Recall the intuition that certain states are not affected by rotation, because they are already aligned with that axis. Rotating $| 0 \rangle$ or $| 1 \rangle$ around the Z-axis ($Z|0\rangle$) does not change the state. Superpositions however are affected by the Z-gate:

In [5]:
Z @ zero_state

array([1.+0.j, 0.+0.j])

The identity gate will returns the state unchanged:

In [6]:
I @ zero_state

array([1.+0.j, 0.+0.j])

# Requirements of quantum logic gates

The main requirement for a quantum logic gates is that the matrix must be unitary. This means that if multiply the matrix by its conjugate transpose, we should get the identity matrix. Recall that the conjugate transpose is just flipping the complex part of the numbers and transposing it. For any gate $U$ we have:

$$
U U^\dagger = I
$$



Let's take for example the Y-gate:

$$
Y = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix}
$$

$$
Y^\dagger = \begin{bmatrix}
0 & i \\
-i & 0
\end{bmatrix}
$$

$$
Y Y^\dagger = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix} \begin{bmatrix}
0 & i \\
-i & 0
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix} = I
$$


In [7]:
y_mul = Y @ Y.conj().T
y_mul

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [8]:
np.allclose(y_mul, I)

True

Another way to frame this requirement is that the computation of evolving a state through a gate must be reversible. If we apply a gate we can get back the original state by applying the conjugate transpose of the gate. 

We'll implement a gate object that check unitarity and some basic properties. For example, the gate must be a square 2D matrix.

To evolve a state through a gate we perform a vector-matrix multiplication with `@`. To get back the original state we apply the conjugate transpose of the gate (`gate.conj().T`) with `@`.

In [9]:
class BaseGate(np.ndarray):
    """Quantum gate class that checks requirements and allows us to encode/decode states."""

    def __new__(cls, input_array):
        arr = np.asarray(input_array, dtype=complex)
        obj = arr.view(cls)
        assert len(obj.shape) == 2  # Quantum gate is a 2D matrix
        assert obj.shape[0] == obj.shape[1]  # Quantum gate is square
        assert np.allclose(obj @ obj.conj().T, np.eye(obj.shape[0]))  # Quantum gate is unitary
        return obj

    def encodes(self, x):
        return np.array(self @ x, dtype=complex)

    def decodes(self, x):
        return np.array(self.conj().T @ x, dtype=complex)

    def __call__(self, x):
        return self.encodes(x)

This allow us to conveniently create gates and use them as blocks to build a circuit.

In [10]:
class X(BaseGate):
    def __new__(cls):
        return super().__new__(cls, np.array([[0, 1], 
                                              [1, 0]], dtype=complex))


class Y(BaseGate):
    def __new__(cls):
        return super().__new__(cls, np.array([[0, -1j], 
                                              [1j, 0]], dtype=complex))


class Z(BaseGate):
    def __new__(cls):
        return super().__new__(cls, np.array([[1, 0], 
                                              [0, -1]], dtype=complex))

In [11]:
x_gate, y_gate, z_gate = X(), Y(), Z()
x_gate(zero_state)

array([0.+0.j, 1.+0.j])

In [12]:
x_gate.decodes(x_gate(zero_state))

array([1.+0.j, 0.+0.j])

To simplify creating circuit we create a `Circuit` class that applies multiple gates in sequence. It can also be used to decode a state back to the original state through the full circuit.

In [13]:
class Circuit(list):
    """Combine multiple gates in sequence."""

    def encodes(self, x):
        for gate in self:
            x = gate.encodes(x)
        return x

    def decodes(self, x):
        for gate in reversed(self):
            x = gate.decodes(x)
        return x
    
    def __call__(self, x): return self.encodes(x)

In [14]:
pipe = Circuit([y_gate, z_gate])

This is our first quantum circuit with multiple different gates. This circuit first flip the state around the Y-axis and then around the Z-axis.

$$
|0\rangle \rightarrow Y \rightarrow Z \rightarrow -i |1\rangle
$$

$$
|0\rangle = \begin{bmatrix}
1 \\
0
\end{bmatrix}
$$

$$
Y |0\rangle = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix} \begin{bmatrix}
1 \\
0
\end{bmatrix} = \begin{bmatrix}
0 \\
i
\end{bmatrix}
$$

$$
Z (Y |0\rangle) = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix} \begin{bmatrix}
0 \\
i
\end{bmatrix} = \begin{bmatrix}
0 \\
-i
\end{bmatrix} = -i |1\rangle
$$

In [15]:
# Z @ Y @ |0>
output_state = pipe(zero_state)
output_state

array([0.+0.j, 0.-1.j])

The decoding is computed as follows:

$$
-i|1\rangle \rightarrow Z^\dagger \rightarrow Y^\dagger \rightarrow |0\rangle
$$

$$
Z^\dagger | \Psi \rangle = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix} \begin{bmatrix}
0 \\
-i
\end{bmatrix} = \begin{bmatrix}
0 \\
i
\end{bmatrix}
$$

$$
Y^\dagger (Z^\dagger | \Psi \rangle) = \begin{bmatrix}
0 & i \\
-i & 0
\end{bmatrix} \begin{bmatrix}
0 \\
i
\end{bmatrix} = \begin{bmatrix}
1 \\
0
\end{bmatrix} = |0\rangle
$$

In [16]:
# Decoding output yields back the original state
pipe.decodes(output_state)

array([1.+0.j, 0.+0.j])

Now we can easily evolve a state through multiple gates one of the few gaps keeping us from building serious quantum algorithms is the generalization to multi qubits. We will discuss a gate involving multiple qubits in the next section on the Clifford group of quantum gates. The generalization of quantum gates to multiple qubits is discussed in-depht in the next notebook. 

## Clifford Group

The Clifford group are a set of common logic gates that are available on any modern quantum computer and are the fundamental building blocks for many quantum algorithms. Most quantum error correction schemes focus on the Clifford group. The Pauli gates are part of the Clifford group. Additionally, the Hadamard (H) gate, S gate and CNOT gates are in the Clifford group of gates. 

Formally, a Clifford gate is any gate where

$$
C P C^\dagger = P
$$

where $C$ is a Clifford gate and $P$ is a Pauli gate. In other words, multiplying a Clifford gate with a Pauli gate and the conjugate transpose of that Clifford gate should yield the original Pauli gate.

For example, let's take $Y$ as our Clifford gate ($C$) and $Z$ as our Pauli gate ($P$).

$$
Y = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix}, \quad
Z = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix}, \quad
Y^\dagger = \begin{bmatrix}
0 & i \\
-i & 0
\end{bmatrix}
$$

$$
Y Z Y^\dagger = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix} \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix} \begin{bmatrix}
0 & i \\
-i & 0
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix} = Z
$$

Therefore it holds that the Y gate is a valid Clifford gate. Lets discuss all gates that are in this unique group.

### Hadamard Gate
The Hadamard gate is an essential gate in quantum computing. It puts qubits in superposition. The reason why this is important in that it is often the 1st step in many quantum algorithms. Very often all qubits are converted into a superposition state, so we can leverage the power of quantum interference. As we will see later, these interference states are essential for quantum parallelism.

$$
H = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}
$$


In [17]:
class H(BaseGate):
    def __new__(cls):
        return super().__new__(cls, np.array([[1, 1], 
                                              [1, -1]] / np.sqrt(2), dtype=complex))

H()(zero_state)

array([0.70710678+0.j, 0.70710678+0.j])

This gate allows you to convert $|0\rangle \rightarrow \frac{1}{\sqrt{2}} (|0\rangle + |1\rangle)$ (i.e. fair coin flip). 

$$
H |0\rangle = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix} \begin{bmatrix}
1 \\
0
\end{bmatrix} = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 \\
1
\end{bmatrix} = \frac{1}{\sqrt{2}} (|0\rangle + |1\rangle)
$$

In [18]:
# Probabilities for H|0> are a fair coin flip
np.abs(H()(zero_state)**2)


array([0.5, 0.5])

### S Gate

The S gate is similar to the Z gate in that it is a phase-shift gate. In fact, it is the square root of the Z gate.

### CX (CNOT) Gate

### Clifford Recap

To recap, here is an overview of all the gates in the Clifford group:

$$
X = \sigma_x = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix},
\quad
Y = \sigma_y = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix},
\quad
Z = \sigma_z = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix},
\quad
I = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$

$$
H = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}, \quad
S = \begin{bmatrix}
1 & 0 \\
0 & i
\end{bmatrix}, \quad
CX = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$


# TODO Explain gates and limitations

## T-Gate

# TODO Explain how the Clifford group and T Gate allows us to approximate any state.

# Generalized Rotation

# TODO Explain General Rotation (R3) gates.

# Work In Progress (WIP)